In [ ]:
# This has been trained inside img_align_celeba which is a big file

In [6]:
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import img_to_array, load_img


In [7]:


# Load attribute data
attr_df = pd.read_csv('list_attr_celeba.csv')
attr_df['image_id'] = attr_df['image_id'].astype(str)

# Directory containing images
img_dir = 'img_align_celeba'  # Replace with your image directory

# Define function to load and preprocess images
def load_and_preprocess_images(attr_df, img_dir, target_size=(224, 224)):
    X = []
    y = []
    for idx, row in attr_df.iterrows():
        img_id = row['image_id']
        img_path = f'{img_dir}/{img_id}'
        
        # Read image
        img = cv2.imread(img_path)
        if img is None:
            continue  # Skip if image cannot be read
        
        # Resize image
        img = cv2.resize(img, target_size)
        
        # Normalize pixel values (assuming RGB images)
        img = img / 255.0
        
        X.append(img)
        
        # Extract attribute labels
        attr_labels = row.iloc[1:].values.astype(int)  # Skip 'image_id' column
        attr_labels[attr_labels == -1] = 0  # Replace -1 with 0
        y.append(attr_labels)
    
    return np.array(X), np.array(y)

# Load and preprocess images
X, y = load_and_preprocess_images(attr_df[:1000], img_dir)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [4]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze ResNet50 layers
base_model.trainable = False

# Function to create model for a single attribute
def create_model():
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    output = layers.Dense(1, activation='sigmoid')(x)  # Output layer for binary classification
    
    model = models.Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Create 40 models, one for each attribute
models_dict = {}
for col in attr_df.columns[1:]:  # Iterate over attribute columns
    model = create_model()
    models_dict[col] = model

# Train each model separately
histories = {}
for col, model in models_dict.items():
    y_train_col = y_train[:, attr_df.columns[1:].tolist().index(col)]
    y_val_col = y_val[:, attr_df.columns[1:].tolist().index(col)]
    
    history = model.fit(X_train, y_train_col,
                        validation_data=(X_val, y_val_col),
                        epochs=1,
                        batch_size=32)
    
    histories[col] = history



20/20 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.8530 - loss: 0.5372 - val_accuracy: 0.9062 - val_loss: 0.3139
20/20 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.6436 - loss: 0.9125 - val_accuracy: 0.6812 - val_loss: 0.6323
20/20 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.5080 - loss: 0.8305 - val_accuracy: 0.4375 - val_loss: 0.7033
20/20 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.6725 - loss: 0.7047 - val_accuracy: 0.8000 - val_loss: 0.5182
20/20 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.9699 - loss: 0.2221 - val_accuracy: 0.9812 - val_loss: 0.0941
20/20 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.8614 - loss: 0.5356 - val_accuracy: 0.8250 - val_loss: 0.5121
20/20 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.7234 - loss: 0.6950 - val_accuracy: 0.6562 - val_loss: 0.6457
20/20 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.6207 - loss: 0.7734 - val_accuracy: 0.7875 - val_loss: 0.5260
20/20 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.6440 - loss: 0.6763 - val_a

In [7]:
# Assume models_dict contains all the trained models for each attribute
for col, model in models_dict.items():
    # Save each model to a file named after the corresponding column
    model.save(f'{col}_model.keras')


In [2]:
import os
import tensorflow as tf

# Directory containing the saved models
models_dir = os.getcwd()  # Replace with your directory containing the models

# Dictionary to store the loaded models
loaded_models = {}
cnt = 0
# Iterate through the files in the directory
for filename in os.listdir(models_dir):
    if filename.endswith('.keras'):
        cnt = cnt+1
        # Extract the model name from the filename (without the extension)
        model_name = os.path.splitext(filename)[0]
        # Load the model
        model_path = os.path.join(models_dir, filename)
        loaded_models[model_name] = tf.keras.models.load_model(model_path)

        if(cnt == 7):
            break 

# Now, loaded_models contains all the models loaded from the directory


C:\Users\hp\AppData\Roaming\Python\Python311\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 434 variables whereas the saved optimizer has 10 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [12]:
# Example: Predict attributes for a single image using loaded models
img_dir = "img_align_celeba"
sample_img_path = f'{img_dir}/{attr_df.iloc[0]["image_id"]}'
sample_img = load_img(sample_img_path, target_size=(224, 224))
sample_img = img_to_array(sample_img)
sample_img = np.expand_dims(sample_img / 255.0, axis=0)  # Normalize and add batch dimension

sample_preds = []

for col, loaded_model in loaded_models.items():
    sample_pred = loaded_model.predict(sample_img)
    sample_preds.append(sample_pred[0][0])

print("Predictions for Sample Image:", sample_preds)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Predictions for Sample Image: [0.07799012, 0.24369387, 0.44508985, 0.2881811, 0.009382524, 0.30842838, 0.38269156]


In [15]:
import numpy as np
sample_preds = np.array(sample_preds)


Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)



In [18]:
threshold = 0.5

# Convert probabilities to binary predictions
binary_pred = (sample_preds >= threshold).astype(int) * 2 - 1

# Attribute descriptions
attribute_descriptions = [
    "5_o_Clock_Shadow", "Arched_Eyebrows", "Attractive", "Bags_Under_Eyes", "Bald", "Bangs", "Big_Lips",
]
# print(binary_pred)
# Describe the attributes present in the sample_pred
descriptions = []
for i, pred in enumerate(binary_pred):
    if pred == 1:
        descriptions.append(f"{attribute_descriptions[i]}: Yes")
    else:
        descriptions.append(f"{attribute_descriptions[i]}: No")

# Print the descriptions
for desc in descriptions:
    print(desc)


[-1 -1 -1 -1 -1 -1 -1]
5_o_Clock_Shadow: No
Arched_Eyebrows: No
Attractive: No
Bags_Under_Eyes: No
Bald: No
Bangs: No
Big_Lips: No
